In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score

np.random.seed(42)

In [3]:
n_samples = 6000

y = np.zeros(n_samples)
y[:120] = 1
np.random.shuffle(y)

X = pd.DataFrame({
    "feature_1": np.random.normal(50, 10, n_samples),
    "feature_2": np.random.normal(30, 5, n_samples),
    "feature_3": np.random.normal(100, 20, n_samples),
    "feature_4": y
})

df = X.copy()
df["target"] = y

print("Dataset shape:", df.shape)
print("\nTarget distribution:")
print(df["target"].value_counts())
df.head()

Dataset shape: (6000, 5)

Target distribution:
target
0.0    5880
1.0     120
Name: count, dtype: int64


,feature_1,feature_2,feature_3,feature_4,target
0,23.509005,30.471488,112.233421,0.0,0.0
1,63.515029,20.536776,82.508700,0.0,0.0
2,59.117653,37.428296,77.176741,0.0,0.0
3,32.666161,25.325802,75.218541,0.0,0.0
4,29.351145,34.765768,82.418838,0.0,0.0


In [4]:
X = df.drop("target", axis=1)
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 4800 samples
Test set: 1200 samples


In [5]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Original Model Accuracy:", accuracy)

Original Model Accuracy: 1.0


## Problem Identified

The accuracy looks extremely high, which immediately raises suspicion. After reviewing the data generation code, I found the issue: feature_4 is identical to the target variable. This is called target leakage.

When creating the features, the code does this:

```python
"feature_4": y
```

This means the model has direct access to the answer during training. It's like giving students the answer key before a test. The model isn't learning patterns, it's just copying feature_4 to make predictions.

In [6]:
print("Checking for target leakage:")
print(f"\nAre feature_4 and target identical? {(df['feature_4'] == df['target']).all()}")
print(f"\nCorrelation between feature_4 and target: {df['feature_4'].corr(df['target']):.4f}")

Checking for target leakage:

Are feature_4 and target identical? True

Correlation between feature_4 and target: 1.0000


## Corrected Implementation

The fix is simple: remove feature_4 since it contains leaked information. The other three features are just random noise with no actual relationship to the target.

In [7]:
X_clean = df.drop(["target", "feature_4"], axis=1)
y = df["target"]

print("Cleaned features:")
print(X_clean.columns.tolist())
print(f"\nShape: {X_clean.shape}")

Cleaned features:
['feature_1', 'feature_2', 'feature_3']

Shape: (6000, 3)


In [8]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(
    X_clean, y, test_size=0.2, random_state=42
)

model_clean = RandomForestClassifier(n_estimators=100, random_state=42)
model_clean.fit(X_train_clean, y_train_clean)

y_pred_clean = model_clean.predict(X_test_clean)

## Proper Evaluation

Now let's evaluate the model correctly without the leaked feature. The dataset is imbalanced (only 2% positive class), so accuracy alone is misleading. I'll look at precision, recall, and the confusion matrix.

In [9]:
accuracy_clean = accuracy_score(y_test_clean, y_pred_clean)
precision = precision_score(y_test_clean, y_pred_clean, zero_division=0)
recall = recall_score(y_test_clean, y_pred_clean)
f1 = f1_score(y_test_clean, y_pred_clean)

print("Corrected Model Performance:")
print(f"Accuracy:  {accuracy_clean:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall:    {recall:.3f}")
print(f"F1-score:  {f1:.3f}")

Corrected Model Performance:
Accuracy:  0.980
Precision: 0.000
Recall:    0.000
F1-score:  0.000


In [10]:
cm = confusion_matrix(y_test_clean, y_pred_clean)

print("\nConfusion Matrix:")
print(cm)
print(f"\nTrue Negatives:  {cm[0,0]}")
print(f"False Positives: {cm[0,1]}")
print(f"False Negatives: {cm[1,0]}")
print(f"True Positives:  {cm[1,1]}")


Confusion Matrix:
[[1176    0]
 [  24    0]]

True Negatives:  1176
False Positives: 0
False Negatives: 24
True Positives:  0


In [11]:
print("\nDetailed Classification Report:")
print(classification_report(y_test_clean, y_pred_clean, target_names=['Negative', 'Positive']))


Detailed Classification Report:
              precision    recall  f1-score   support

    Negative       0.98      1.00      0.99      1176
    Positive       0.00      0.00      0.00        24

    accuracy                           0.98      1200
   macro avg       0.49      0.50      0.49      1200
weighted avg       0.96      0.98      0.97      1200



c:\Users\aakif\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aakif\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\aakif\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Explanation

The original model had near-perfect accuracy because it was cheating. Feature_4 was identical to the target, so the model just learned to copy that value.

After removing the leaked feature, the model predicts only the negative class (majority class) because the remaining features are just random noise with no real relationship to the target. The high accuracy we see now (around 98%) is misleading for a different reason: it's simply predicting the majority class most of the time.

With only 2% positive samples, a model that always predicts negative would get 98% accuracy. Looking at the confusion matrix and recall shows the model isn't actually learning anything useful.

This demonstrates why accuracy alone is a poor metric for imbalanced datasets and why we must carefully check for data leakage during model development.